In [1]:
import os
import re
from pathlib import Path
import numpy as np
from patchify import patchify
from PIL import Image

In [2]:
def create_folder():
    FOLDERS = ['train', 'val', 'test']
    for folder in FOLDERS:
        if not os.path.exists(folder):          
            folder_imgs = f"{folder}/images"
            folder_masks = f"{folder}/masks"      
            
            if not os.path.exists(folder_imgs):
                os.makedirs(folder_imgs)
                
            if not os.path.exists(folder_masks):
                os.makedirs(folder_masks)

In [3]:
create_folder()

In [4]:
def create_patches(src_image, dest_path):
    path_split = os.path.split(src_image)
    tile_num = re.findall(r'\d+', path_split[0])[0]
    
    image = Image.open(src_image)
    image = np.asarray(image)
    if len(image.shape) > 2:  # only if color channel exists as well
#         patches = patchify(image, (320, 320, 3), step=300)
        patches = patchify(image, (256,256,3), step=256)
    
        file_name_wo_ext = Path(src_image).stem
        for i in range(patches.shape[0]):
            for j in range(patches.shape[1]):
                patch = patches[i, j, 0]
                patch = Image.fromarray(patch)
                num = i * patches.shape[1] + j
                patch.save(f"{dest_path}/{file_name_wo_ext}_tile_{tile_num}_patch_{num}.png")

In [5]:
for path_name, _, file_name in os.walk('data'):
    for f in file_name:
        if f != 'classes.json':
            
            path_split = os.path.split(path_name)
            tile_num = re.findall(r'\d+', path_split[0])[0]
            
            img_type =path_split[1]  
            
            # leave out tile 2, issues with color dim
            if tile_num == '3':
                target_folder_imgs = 'val'
                target_folder_masks = 'val'
            elif tile_num == '1':
                target_folder_imgs = 'test'
                target_folder_masks = 'test'
            elif tile_num in ['4', '5', '6', '7', '8']:
                target_folder_imgs = 'train'
                target_folder_masks = 'train'
            
            # copy all images
            src_image = os.path.join(path_name, f)
            file_name_wo_ext = Path(src_image).stem
            # check if file exists in images and masks
            img_file = f"{path_split[0]}/images/{file_name_wo_ext}.jpg"
            mask_file = f"{path_split[0]}/masks/{file_name_wo_ext}.png"
            if os.path.exists(img_file) and os.path.exists(mask_file):
                if img_type == 'images':
                    dest = os.path.join(target_folder_imgs, img_type)
                    create_patches(src_image=src_image, dest_path=dest)        
                
                # copy all masks
                if img_type == 'masks':
                    dest = os.path.join(target_folder_masks, img_type)
                    create_patches(src_image=src_image, dest_path=dest)

<h4> Dataset </h4>

In [27]:
from torch.utils.data import Dataset
import os
from pathlib import Path
import torchvision.transforms as transforms

In [7]:
path_name = "./test"
image_names = os.listdir(f"{path_name}/images")

In [8]:
image_paths = [f"{path_name}/images/{i}" for i in image_names]

In [9]:
# Function to convert hexadecimal color code to grayscale intensity value
def hex_to_intensity(hex_color):
    # Convert hexadecimal color to RGB
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (1, 3, 5))  # Extract RGB components

    # Calculate intensity value (grayscale) from RGB using luminance formula
    intensity = 0.2989 * rgb[0] + 0.5870 * rgb[1] + 0.1140 * rgb[2]
    
    return intensity

# Hexadecimal color codes for classes
colors = {
    'Building': '#3C1098',
    'Land': '#8429F6',
    'Road': '#6EC1E4',
    'Vegetation': '#FEDD3A',
    'Water': '#E2A929',
    'Unlabeled': '#9B9B9B'
}

# Calculate intensity values for each class color
for class_name, hex_code in colors.items():
    intensity = hex_to_intensity(hex_code)
    print(f"{class_name}: {intensity:.2f}")


Building: 44.65
Land: 91.57
Road: 172.16
Vegetation: 212.26
Water: 171.43
Unlabeled: 154.98


In [30]:
class SegmentationDataset(Dataset):
    
    def __init__(self, path_name):    #pathname refers to either train, test, val
        super().__init__()
        self.image_names = os.listdir(f"{path_name}/images")
        #image_part_001_tile_1_patch_0.png
        
        self.image_paths = [f"{path_name}/images/{i}" for i in self.image_names]
        #'./test/images/image_part_001_tile_1_patch_0.png'
        
        self.mask_names = os.listdir(f"{path_name}/masks")
        self.mask_paths = [f"{path_name}/masks/{i}" for i in self.mask_names]
        
#exclude images that don't exist in both folder-------------------
        self.img_stem = [Path(i).stem for i in self.image_paths]
        #image_part_001_tile_1_patch_0
        
        self.mask_stem = [Path(i).stem for i in self.mask_paths]
        self.img_mask_stem = set(self.img_stem) & set(self.mask_stem)
        self.image_paths = [i for i in self.image_paths if (Path(i).stem in self.img_mask_stem)]
    
    def __len__(self):
        return len(self.img_mask_stem)
    
    def convert_mask(self, mask):            #obtained after converting RGB to gray-scaled from kaggle dataset.
        mask[mask == 155] = 0  # unlabeled
        mask[mask == 44] = 1  # building
        mask[mask == 91] = 2  # land
        mask[mask == 171] = 3  # water
        mask[mask == 172] = 4  # road
        mask[mask == 212] = 5  # vegetation
        return mask   
    
    def __getitem__(self, index):
        image = Image.open(self.image_paths[index])
        transform = transforms.ToTensor()
        image = transform(image)
        
        mask = Image.open(self.mask_paths[index]).convert("L")    #to grayscale image
        mask = transform(mask)
        mask = self.convert_mask(mask)
        
        return image, mask

### Model 

In [40]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
EPOCHS = 5
BS = 4

In [43]:
train_ds = SegmentationDataset("./train")
train_dataloader = DataLoader(train_ds, batch_size = BS, shuffle = True)

val_ds = SegmentationDataset("./val")
val_dataloader = DataLoader(val_ds, batch_size = BS, shuffle = True)

In [46]:
train_dataloader.shape()

AttributeError: 'DataLoader' object has no attribute 'shape'

In [53]:
next(iter(train_dataloader))[0].shape

torch.Size([4, 3, 256, 256])